<a href="https://colab.research.google.com/github/parisazeynaly/BigData_neo4j/blob/main/Fraud_Detection_in_P2P_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df=pd.read_csv('training.1600000.processed.noemoticon.csv', nrows=10000, encoding='latin-1', header=None)

In [ ]:
print(df.head(10))
print(f"\nTotal rows loaded: {len(df)}")

   0           1                             2         3                4  \
0  0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY  _TheSpecialOne_   
1  0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY    scotthamilton   
2  0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY         mattycus   
3  0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY          ElleCTF   
4  0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY           Karoli   
5  0  1467811372  Mon Apr 06 22:20:00 PDT 2009  NO_QUERY         joy_wolf   
6  0  1467811592  Mon Apr 06 22:20:03 PDT 2009  NO_QUERY          mybirch   
7  0  1467811594  Mon Apr 06 22:20:03 PDT 2009  NO_QUERY             coZZ   
8  0  1467811795  Mon Apr 06 22:20:05 PDT 2009  NO_QUERY  2Hood4Hollywood   
9  0  1467812025  Mon Apr 06 22:20:09 PDT 2009  NO_QUERY          mimismo   

                                                   5  
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1  is upset that he can't update his Faceb

In [ ]:
df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']


preprocessing


In [ ]:
def clean_text(text):
  text= text.lower()
  text= re.sub(r"i'm","i am",text)
  text= re.sub(r"he's","he is",text)
  text= re.sub(r"she's","she is",text)
  text=re.sub(r'@\w+', '', text)  # remove mentions
  text = re.sub(r'#\w+', '', text)  # remove hashtags
  text = re.sub(r'[^a-z\s]', '', text)  # remove non-letters
  text = re.sub(r'\s+', ' ', text).strip()  # clean extra spaces
  return text

In [ ]:
df['clean_text']=df.apply(lambda row: clean_text(row['text']), axis=1)

In [ ]:
def extract_mentions(text):
   return re.findall(r'@(\w+)',text)

In [ ]:
df['mention']= df['text'].apply(extract_mentions)

In [ ]:
df['mentions'] = df['text'].apply(extract_mentions)

# Preview the result
print(df[['user', 'text', 'clean_text', 'mentions']].head(10))

              user                                               text  \
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1    scotthamilton  is upset that he can't update his Facebook by ...   
2         mattycus  @Kenichan I dived many times for the ball. Man...   
3          ElleCTF    my whole body feels itchy and like its on fire    
4           Karoli  @nationwideclass no, it's not behaving at all....   
5         joy_wolf                      @Kwesidei not the whole crew    
6          mybirch                                        Need a hug    
7             coZZ  @LOLTrish hey  long time no see! Yes.. Rains a...   
8  2Hood4Hollywood               @Tatiana_K nope they didn't have it    
9          mimismo                          @twittera que me muera ?    

                                          clean_text           mentions  
0  httptwitpiccomyzl awww thats a bummer you shou...       [switchfoot]  
1  is upset that he cant update his facebook by 

In [ ]:
edges = []

for _, row in df.iterrows():
    source_user = row['user']
    sentiment = int(row['target'])
    mentioned_users = extract_mentions(row['text'])

    for target_user in mentioned_users:
        edges.append({
            'source': source_user,
            'target': target_user,
            'sentiment': 'positive' if sentiment == 4 else 'negative'
        })

edges_df = pd.DataFrame(edges)
edges_df.head()


,source,target,sentiment
0,_TheSpecialOne_,switchfoot,negative
1,mattycus,Kenichan,negative
2,Karoli,nationwideclass,negative
3,joy_wolf,Kwesidei,negative
4,coZZ,LOLTrish,negative


In [ ]:
edges_df.to_csv("mentions.csv", index=False)